In [1]:
import torch
from torch import nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import MNIST # Training dataset
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# obtain mnist data and process
batch_size = 128
num_of_classes = 10

def get_indices(dataset,ind_array):
    indices =  []
    for i in range(len(dataset.targets)):
        for ind in ind_array:
            if dataset.targets[i] == ind:
                indices.append(i)
    return indices

dataset = MNIST('.', train=True, download=False,transform=transforms.ToTensor())

idx = get_indices(dataset, np.arange(num_of_classes))
data_loader_target = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))
data_loader_gan = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))
data_loader_test = DataLoader(dataset, batch_size=batch_size, sampler = SubsetRandomSampler(idx))

classes = np.arange(num_of_classes)

In [3]:
# target model
from net_conv import target_net

net = target_net(num_of_classes)
criterion_tar = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

PATH = './target_models/basic_net_convolutional_'+str(num_of_classes)+'classes.pth'
# train and and save the model
#net.train(data_loader_target, criterion_tar, optimizer)
#torch.save(net.state_dict(), PATH)
# load the model
net = target_net(num_of_classes)
net.load_state_dict(torch.load(PATH))

print('model accuracy: ', net.accuracy(data_loader_gan))

model accuracy:  98.38


In [4]:
from advGAN_conv import advGAN
advgan = advGAN(net,gen_path_extra='basic_advgan')

In [ ]:
# train the gen/disc
gen,disc = advgan.train(data_loader_gan)

/home/elacker/Desktop/ExML/exmlenv/lib64/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


In [ ]:
# load the generator 
gen = advgan.load_gen()

In [ ]:
from sklearn.metrics import accuracy_score

def one_batch(data,num=1):
    for i,(img,lbl) in enumerate(data):
        if i == num:
            return img,lbl
            break

images,labels = one_batch(data_loader_test,num=1)
perts = gen(images.reshape(len(images),28*28))
adv_images = images.reshape(len(images),28*28) + perts
net_out = net(images.reshape(len(images),28*28))
adv_pred = net(adv_images)

print('% wrong: ',1.-accuracy_score(labels,torch.argmax(adv_pred,dim=1)))
print('-----------------------------------------')
for img,lbl,adimg,out,adpred,p in zip(images,labels,adv_images,net_out,adv_pred,perts):
    print('target model output: ', int(torch.argmax(out).detach()))
    print('target model output on adv example: ', int(torch.argmax(adpred).detach()))
    print('frobenius norm of pertubation: ',float(torch.norm(p,'fro').detach()))
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle('actual label: '+str(int(lbl.detach())))
    ax1.imshow(img.reshape(28,28).detach().numpy(),cmap='gray')
    ax2.imshow(adimg.reshape(28,28).detach().numpy(),cmap='gray')
    plt.show()
